# Big-Data

Description

In [6]:
# Install necessary libraries to run this IPython Notebook
!pip install folium

## Treatment of data

In [7]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap

In [8]:
# Load spreadsheet
spreadsheet = pd.ExcelFile('datasets/imd_student_blind.xlsx')

# Load a sheet into a DataFrame by index
imd_student_blind_df = spreadsheet.parse(0)

In [9]:
# Replace 'disciplina_ID' for its abreviation name

classes_map = {
    0: 'ITP',
    1: 'PLE1',
    2: 'RPMTI',
    3: 'CDI1',
    4: 'PLE2',
    5: 'FMC1',
    6: 'VGA' 
}

imd_student_blind_df['disciplina_ID'].replace(classes_map, inplace = True)

In [10]:
# Print the head of the DataFrame df
imd_student_blind_df.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,enem-matematica,enem-redacao,enem-nota
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,ITP,Reprovado,2172,640,618.0
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,ITP,Aprovado,2172,640,618.0
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,ITP,Reprovado,1955,660,615.0
3,2,59072580,2014,1,ATIVO,2014,2,6.1,ITP,Aprovado,1959,560,600.0
4,3,59088150,2014,1,ATIVO,2014,1,3.0,ITP,Reprovado,2235,760,673.0


In [11]:
# Print the columns' name
imd_student_blind_df.columns

Index(['a_ID', 'CEP', 'ano_ingresso', 'periodo_ingresso', 'status',
       'ano_disciplina', 'periodo_disciplina', 'nota', 'disciplina_ID',
       'status.disciplina', 'enem-matematica', 'enem-redacao', 'enem-nota'],
      dtype='object')

In [12]:
imd_student_blind_df.shape

(4842, 13)

In [13]:
# Load geolocation extracted data
students_geolocation = pd.read_csv('datasets/students_geolocation.csv')

In [14]:
students_geolocation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 7 columns):
id               900 non-null int64
zipcode          900 non-null int64
total_entries    900 non-null int64
lat              736 non-null float64
lng              736 non-null float64
address          736 non-null object
distance         736 non-null float64
dtypes: float64(3), int64(3), object(1)
memory usage: 49.3+ KB


In [15]:
students_geolocation.head()

,id,zipcode,total_entries,lat,lng,address,distance
0,0,59015430,12,-5.816641,-35.200015,"Nova Descoberta, Natal - RN, 59015-430, Brazil",1.815783
1,1,59073120,6,-5.853337,-35.252804,"Planalto, Natal - RN, 59073-120, Brazil",5.749127
2,2,59072580,12,-5.832998,-35.242542,"Cidade Nova, Natal - RN, 59072-580, Brazil",4.114743
3,3,59088150,9,-5.872282,-35.206600,"Neópolis, Natal - RN, 59088-150, Brazil",4.439972
4,4,59064245,9,NaN,NaN,NaN,NaN


In [16]:
grouped_df = imd_student_blind_df.groupby('a_ID')

students_summary_df = pd.DataFrame({
        'grades_mean': grouped_df['nota'].mean(),
        'grades_amount': grouped_df['nota'].count(),
        'enem_rating': grouped_df['enem-nota'].mean(),
        'enem_math': grouped_df['enem-matematica'].mean(),
        'enem_wording': grouped_df['enem-redacao'].mean(),
    }).reset_index()

students_summary_df.rename(
    columns = {
        'a_ID' : 'id'
    },
    inplace = True
)

students_summary_df['distance'] = students_geolocation[students_geolocation['id'] == students_summary_df['id']]['distance']
students_summary_df['lat'] = students_geolocation[students_geolocation['id'] == students_summary_df['id']]['lat']
students_summary_df['lng'] = students_geolocation[students_geolocation['id'] == students_summary_df['id']]['lng']

In [17]:
students_summary_df.head()

,id,enem_math,enem_rating,enem_wording,grades_amount,grades_mean,distance,lat,lng
0,0,2172,618.0,640,12,2.916667,1.815783,-5.816641,-35.200015
1,1,1955,615.0,660,6,3.216667,5.749127,-5.853337,-35.252804
2,2,1959,600.0,560,12,4.116667,4.114743,-5.832998,-35.242542
3,3,2235,673.0,760,9,5.588889,4.439972,-5.872282,-35.206600
4,4,2132,642.0,700,9,3.422222,NaN,NaN,NaN


In [18]:
grades_x_distance = pd.DataFrame({
    'distance': students_summary_df['distance'],
    'grades_mean': students_summary_df['grades_mean']
})

# Drop NaN values
grades_x_distance.dropna(inplace = True)

# Get only entries that have less than 30km from distance
grades_x_distance = grades_x_distance[grades_x_distance['distance'] < 30]

In [19]:
grades_x_distance.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568 entries, 0 to 899
Data columns (total 2 columns):
distance       568 non-null float64
grades_mean    568 non-null float64
dtypes: float64(2)
memory usage: 13.3 KB


## Analysis

In [20]:
# Import modules from Bokeh visualization library

from bokeh.charts import Donut, Scatter, BoxPlot, Bar
from bokeh.layouts import gridplot
from bokeh.io import output_notebook, show
from bokeh.plotting import *

In [21]:
imd_student_blind_df[['a_ID', 'ano_ingresso']].drop_duplicates().groupby('ano_ingresso').count()

#899

,a_ID
ano_ingresso,
2014,253
2015,320
2016,333


In [22]:
id_by_year = imd_student_blind_df[['a_ID', 'ano_ingresso']].drop_duplicates().groupby('ano_ingresso').count()
id_by_year

p = Bar(id_by_year, values='a_ID', xlabel='Ano', ylabel='Quantidade de estudantes', 
        title="Estudantes por ano", color='a_ID')

output_notebook()
show(p)

Loading BokehJS ...

In [23]:
'''
data_groupby = ['ano_disciplina', 'periodo_disciplina', 'disciplina_ID']
nota = imd_student_blind_df.groupby(data_groupby)['nota']
nota.mean()
'''
# Gráfico
graph_lines = figure(plot_width=900, plot_height=300, title="Média de nota de cada disciplina, por período letivo")
colors = ['orange', 'blue', 'red', 'green', 'yellow', 'pink', 'purple', 'brown']
i = 0

# Array
ano_periodo_select = ['ano_disciplina', 'periodo_disciplina']

# Imprimir linhas de cada disciplina
for disciplina_ID in imd_student_blind_df['disciplina_ID'].unique():
    # Dados da disciplina
    data_disciplina = imd_student_blind_df[imd_student_blind_df['disciplina_ID'] == disciplina_ID]
    # Percorre o ano e o período
    ano_periodo = data_disciplina[ano_periodo_select].drop_duplicates()
    xline = []
    yline = []
    
    for ano in ano_periodo['ano_disciplina'].sort_values().unique():
        periodos_data = data_disciplina[data_disciplina['ano_disciplina'] == ano][['periodo_disciplina', 'nota']].sort_values('periodo_disciplina')
        for periodo in ano_periodo['periodo_disciplina'].sort_values().unique():
            # Se foi no primeiro ou segundo período
            if(periodo == 2):
                periodo_graph = 5
            else:
                periodo_graph = 0
            # X
            ap = "{0}.{1}".format(ano, periodo_graph)
            xline.append(float(ap))
            # Y
            media_periodo = periodos_data[periodos_data['periodo_disciplina'] == periodo]['nota'].mean()
            yline.append(float(media_periodo))
    # Data source para criar a linha
    line_disciplina_ds = ColumnDataSource(
        data=dict(
            x=xline,
            y=yline
        )
    )
    title = '{0}'.format(disciplina_ID)
    graph_lines.line('x', 'y', line_width=3, source=line_disciplina_ds, legend=title, color=colors[i])
    graph_lines.circle(xline, yline, color=colors[i], size=6, alpha=1.0)
    i += 1 # Avança no contador de cor
    
output_notebook()
show(graph_lines)

Loading BokehJS ...

In [24]:
p_grades_x_distance = Scatter(
        grades_x_distance,
        x='grades_mean',
        y='distance', 
        xlabel='Grades Mean',
        ylabel='Distance from IMD in kilometers',
        title='Grades Mean vs Distance from IMD',
        plot_width=800, plot_height=400
)

output_notebook()

show(p_grades_x_distance)

Loading BokehJS ...

In [25]:
# Create a coordinates list with the filtered data and show the heatmap

coordinates = []

for i in range(len(students_summary_df)):
    if all(~np.isnan([students_summary_df.ix[i,'lat'], students_summary_df.ix[i,'lng']])):
        coordinates.append([students_summary_df.ix[i,'lat'], students_summary_df.ix[i,'lng'], students_summary_df.ix[i,'grades_mean']])
        
f_map = folium.Map(
    location = [-5.791659, -35.28385],
    zoom_start = 11
)

HeatMap(coordinates).add_to(f_map)

f_map

### RPMTI dependents subjects analysis

In [26]:
def get_group_by_grade(grade):
    if(grade >= 5 and grade < 6):
        return '5 - 6'
    if(grade >= 6 and grade < 7):
        return '6 - 7'
    if(grade >= 7 and grade < 8):
        return '7 - 8'
    if(grade >= 8 and grade < 9):
        return '8 - 9'
    if(grade >= 9):
        return '9 - 10'

In [27]:
# Add the lib package to the system path, so that we can include SubjectConnections from there.

import os
import sys
module_path = os.path.abspath(os.path.join('./'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from lib.subject_connections import SubjectConnections

In [28]:
RPMTI_student = imd_student_blind_df['disciplina_ID'] == 'RPMTI'
has_approved_status = imd_student_blind_df['status.disciplina'] == 'Aprovado'

RPMTI_students_df = imd_student_blind_df[RPMTI_student & has_approved_status]

RPMTI_connections = SubjectConnections(RPMTI_students_df, 'a_ID')
RPMTI_connections.parse_column('nota', 'grade_group', get_group_by_grade)

In [29]:
def generate_RPMTI_boxplot(df, subject_key): 
    return BoxPlot(
        df,
        values = '{}_grade'.format(subject_key),
        label  = 'grade_group',
        color  = 'grade_group',
        xlabel = 'RPMTI grade groups',
        ylabel = '{} grades'.format(subject_key), 
        title  = '{} student grades (first enrollment) grouped by RPMTI grades'.format(subject_key)
    )

def RPMTI_dependent_subjects(list):
    boxplot_list = []

    for subject in list:
        column_name = '{}_grade'.format(subject)

        students_df = imd_student_blind_df[imd_student_blind_df['disciplina_ID'] == subject]
        df = RPMTI_connections.obtain_values_from(students_df, column_name, ['grade_group'])
        
        grade_groups_mean_df = df.groupby('grade_group')[column_name].mean().reset_index()
        
        box = generate_RPMTI_boxplot(df, subject)
        box.line(x = grade_groups_mean_df['grade_group'], y = grade_groups_mean_df[column_name], line_width = 2, line_color = '#2C3E50')
        box.circle(x = grade_groups_mean_df['grade_group'], y = grade_groups_mean_df[column_name], color = "#F1C40F", size = 5)

        boxplot_list.append(box)
    
    return boxplot_list
    

boxplots = RPMTI_dependent_subjects(['ITP', 'FMC1', 'CDI1', 'VGA'])

grid = gridplot(boxplots, ncols=2, plot_width=460, plot_height=400)

output_notebook()
show(grid)

Loading BokehJS ...

In [30]:
RPMTI_connections.base_df.groupby('grade_group', as_index=False)['a_ID'].agg({
    'amount': 'count'
})

,grade_group,amount
0,5 - 6,187
1,6 - 7,132
2,7 - 8,82
3,8 - 9,61
4,9 - 10,31


In [31]:
ITP_students_df = imd_student_blind_df[imd_student_blind_df['disciplina_ID'] == 'ITP']

In [32]:
# Total enrollments per student in ITP
ITP_enrollments = ITP_students_df.groupby('a_ID', as_index = False)['nota'].agg({
    'total': 'count'
})

ITP_enrollments.groupby('total', as_index = False)['a_ID'].count()

,total,a_ID
0,1,334
1,2,109
2,3,28
3,4,6
4,5,2


## Enem Analysis

In [42]:
grades_x_enem = pd.DataFrame({
    'enem_rating': students_summary_df['enem_rating'],
    'grades_mean': students_summary_df['grades_mean'],
})


# Drop NaN values
grades_x_enem.dropna(inplace = True)

p_grades_x_enem = Scatter(
        grades_x_enem,
        x='grades_mean',
        y='enem_rating', 
        xlabel='Grades Mean',
        ylabel='Enem Rating',
        title='Grades Mean vs Enem Rating',
        plot_width=800, plot_height=400
)

output_notebook()

show(p_grades_x_enem)

Loading BokehJS ...

In [40]:
group_enem_df = grades_x_enem.groupby(pd.cut(grades_x_enem['enem_rating'], np.arange(500, 1.0+900, 100)))



enem = pd.DataFrame({
        'total': (group_enem_df['grades_mean']).count(),
        '0-3': group_enem_df['grades_mean'].filter(lambda x: (x < 3).any()).count(),
        '3-5': group_enem_df['grades_mean'].filter(lambda x: (x > 3 and x < 5).any()).count(),
        '5-7': group_enem_df['grades_mean'].filter(lambda x: (x > 5 and x < 7).any()).count(),
        '7-10': group_enem_df['grades_mean'].filter(lambda x: (x > 7 and x <=10).any()).count(),
    }).reset_index()

TypeError: the filter must return a boolean result

In [35]:
from bokeh.charts import Donut, show, output_file

enem = pd.melt(enem, id_vars=['enem_rating'],
             value_vars=['0-3', '3-5', '5-7', '7-10'],
             value_name='mean_count', var_name='grades_mean')
             
d = Donut(enem, label=['Nota Enem', 'Média de notas'], values= 'mean_count',  text_font_size='8pt', hover_text='mean_count')

show(d)

TypeError: 'NoneType' object is not iterable